In [1]:
from fastai.collab import *
from fastai.tabular.all import *

In [2]:
path = untar_data(URLs.ML_SAMPLE)

In [3]:
path.ls()

(#1) [Path('C:/Users/DANIEL PC/.fastai/data/movie_lens_sample/ratings.csv')]

In [7]:
path

Path('C:/Users/DANIEL PC/.fastai/data/movie_lens_sample')

In [55]:
ratings = pd.read_csv(path/'ratings.csv', delimiter=',')

ratings.head()
len(ratings)

6031

In [58]:
#ratings["userId"].value_counts()
np.array(ratings["userId"].value_counts()).sum()

6031

In [6]:
last_skywalker = np.array([0.98, 0.9, -0.9]).T
last_skywalker.shape

(3,)

In [7]:
user1 = np.array([0.9, 0.8, -0.6])

In [12]:
(last_skywalker@user1)

2.1420000000000003

In [7]:
(user1*last_skywalker).sum()

2.1420000000000003

In [8]:
casablanca = np.array([-0.99, -0.3, 0.8])

In [12]:
user1@last_skywalker

2.1420000000000003

In [13]:
dls = CollabDataLoaders.from_df(ratings, bs=64)

In [14]:
dls.show_batch()

,userId,movieId,rating
0,77,344,3.0
1,585,1923,3.0
2,243,1721,3.5
3,509,527,5.0
4,624,356,3.0
5,460,357,5.0
6,102,2762,4.0
7,311,1291,4.5
8,574,50,5.0
9,19,608,5.0


In [25]:
n_users = len(dls.classes['userId'])
n_movies = len(dls.classes['movieId'])
n_factors = 5

user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

In [62]:
Embedding??

In [29]:
user_factors.shape, movie_factors.shape

(torch.Size([101, 5]), torch.Size([101, 5]))

In [30]:
(user_factors@movie_factors.T).shape

torch.Size([101, 101])

In [51]:
one_hot_3 = one_hot(10, n_users).float()


In [59]:
user_factors.t()@one_hot_3

tensor([ 0.5691,  0.9549, -1.2714,  1.6854,  0.4808])

In [60]:
user_factors[10]

tensor([ 0.5691,  0.9549, -1.2714,  1.6854,  0.4808])

In [20]:
user_factors.t() @ one_hot_3

tensor([ 0.7878, -0.8733, -0.6771, -0.1430,  0.5941])

In [30]:
class Example:
    def __init__(self, a): self.a = a
    def say(self, x): return f"Hello {self.a} {x}"
    


In [13]:
x,y = dls.one_batch()
x.shape

torch.Size([64, 2])

In [21]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_users, 1)
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:, 0]) + self.movie_bias(x[:, 1])
        return sigmoid_range(res, *self.y_range)

In [22]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [23]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.807254,1.319869,00:06
1,0.924677,0.661780,00:00
2,0.673528,0.631958,00:01
3,0.571910,0.618124,00:01
4,0.528694,0.616027,00:01


## To prevent overfitting

### - use data augmentation:
    Regularization

**Weights Decay (L2 Regularization)**

In [24]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,1.804615,1.358706,00:01
1,0.928841,0.666290,00:01
2,0.684787,0.628290,00:01
3,0.596701,0.626152,00:01
4,0.539298,0.624957,00:01


In [25]:
class T(Module):
    def __init__(self):
        self.a = nn.Linear(5, 1)
L(T().parameters())

(#2) [Parameter containing:
tensor([[ 0.1367,  0.2688,  0.0764, -0.1971,  0.1457]], requires_grad=True),Parameter containing:
tensor([-0.3737], requires_grad=True)]

In [36]:
nn.Parameter(torch.zeros(5,9).normal_(0,0.01))

Parameter containing:
tensor([[ 0.0206, -0.0026,  0.0173, -0.0127, -0.0015, -0.0172,  0.0005,  0.0146,
         -0.0005],
        [-0.0070, -0.0156, -0.0091, -0.0038,  0.0017,  0.0050, -0.0014,  0.0125,
          0.0021],
        [-0.0178,  0.0135,  0.0105, -0.0113,  0.0027,  0.0058, -0.0075,  0.0062,
         -0.0075],
        [-0.0079,  0.0191, -0.0210,  0.0007, -0.0134, -0.0187,  0.0161,  0.0020,
          0.0083],
        [-0.0055,  0.0062, -0.0231, -0.0084,  0.0003,  0.0153,  0.0035, -0.0090,
         -0.0055]], requires_grad=True)

In [37]:
def create_params(size):
    return nn.Parameter(torch.zeros(*size).normal_(0,0.01))

In [42]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = create_params([n_users, n_factors])
        self.user_bias = create_params([n_users])
        self.movie_factors = create_params([n_movies, n_factors])
        self.movie_bias = create_params([n_users])
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors[x[:, 0]]
        movies = self.movie_factors[x[:, 1]]
        
        res = (users * movies).sum(dim=1)
        res += self.user_bias[x[:, 0]] + self.movie_bias[x[:, 1]]
        return sigmoid_range(res, *self.y_range)

In [43]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,1.808091,1.394788,00:01
1,0.925537,0.651783,00:00
2,0.676310,0.625235,00:00
3,0.572817,0.617409,00:01
4,0.526014,0.614409,00:01


In [48]:
movie_bias = learn.model.movie_bias.squeeze()
idxs = movie_bias.argsort()[:5]
[dls.classes['movieId'][i] for i in idxs]

[153, 344, 2628, 231, 587]

In [49]:
learn = collab_learner(dls, n_factors=50, y_range=(0,5.5))

In [50]:
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,1.810883,1.355641,00:01
1,0.927284,0.662516,00:01
2,0.666605,0.618508,00:00
3,0.564084,0.607518,00:00
4,0.521363,0.606543,00:01
